In [1]:
import pandas as pd
import yfinance as yf
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from sklearn.preprocessing import MinMaxScaler

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gps_stock_merge = pd.read_csv(r'C:\Users\User\Desktop\thesis_17_3_2022\stock_price_and_news\topic\topic_selection\gps_topic\stock_gps.csv')
gps_manage_merge = pd.read_csv(r'C:\Users\User\Desktop\thesis_17_3_2022\stock_price_and_news\topic\topic_selection\gps_topic\management_gps.csv')
gps_performance_merge = pd.read_csv(r'C:\Users\User\Desktop\thesis_17_3_2022\stock_price_and_news\topic\topic_selection\gps_topic\performance_gps.csv')
gps_sales_merge = pd.read_csv(r'C:\Users\User\Desktop\thesis_17_3_2022\stock_price_and_news\topic\topic_selection\gps_topic\sales_gps.csv')


In [3]:
gps_manage_merge

,Unnamed: 0,Date,text_headlines
0,202,2021-07-08 03:05:00,how to get people to use store credit cards mo...
1,362,2021-02-09 06:27:00,biden and yellen discuss $19 trillion covid re...
2,625,2020-03-05 04:30:00,gap chooses old navy leader as new ceo
3,659,2019-11-09 10:26:00,old navy spinoff thrown into question by the d...
4,662,2019-11-08 07:45:00,gap ceo departure the latest spot of bad news ...
...,...,...,...
115,2880,2007-01-23 11:14:00,ceo pressler's out at gap inc
116,2883,2007-01-22 07:31:00,ti shares rise after results; gap ceo resigns
117,2885,2007-01-22 05:28:00,ceo pressler is out at gap inc
118,2897,2007-01-04 06:01:00,updates advisories and surprises


In [4]:
gps_stock_merge1 = gps_stock_merge['text_headlines']
gps_manage_merge1 = gps_manage_merge['text_headlines']
gps_performance_merge1 = gps_performance_merge['text_headlines']
gps_sales_merge1 = gps_sales_merge['text_headlines']


In [5]:
gps_stock_merge1

0        gap inc stock falls monday underperforms market
1        gap inc stock falls friday underperforms market
2      gap inc stock underperforms thursday when comp...
3      gap inc stock falls wednesday underperforms ma...
4      gap inc stock outperforms market on strong tra...
                             ...                        
502                     updates advisories and surprises
503                gap surprises with strong march sales
504                     updates advisories and surprises
505                     updates advisories and surprises
506                     updates advisories and surprises
Name: text_headlines, Length: 507, dtype: object

In [6]:
gps_stock_merge2 = gps_stock_merge1.to_list()

In [7]:
gps_performance_merge2 =gps_performance_merge1.to_list()
gps_manage_merge2 =gps_manage_merge1.to_list()
gps_sales_merge2 =gps_sales_merge1.to_list()


In [8]:
gps_stock_merge2[-10:]

['updates advisories and surprises',
 'amgen rises after results; gap names new ceo',
 'updates advisories and surprises',
 'updates advisories and surprises',
 'updates advisories and surprises',
 'updates advisories and surprises',
 'gap surprises with strong march sales',
 'updates advisories and surprises',
 'updates advisories and surprises',
 'updates advisories and surprises']

In [9]:
from transformers import BertModel, BertTokenizer,AutoModelForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = 3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis=1)[:, None])
    return e_x / np.sum(e_x, axis=1)[:, None]

In [11]:
label_dict = {0: 'positive', 1: 'negative', 2: 'neutral'}

gps_out_stock = []
gps_score_stock = []
for i in range(0, len(gps_stock_merge2), 10):
    tokenized_gps_stock = tokenizer(gps_stock_merge2[i:i+10], truncation = True, padding = True, return_tensors = "pt")
    gps_output_stock = model(tokenized_gps_stock['input_ids'])
    soft_logits_gps_stock = softmax(np.array(gps_output_stock.logits.detach().cpu()))
    sentiment_score_gps_stock = soft_logits_gps_stock[:,0]-soft_logits_gps_stock[:,1]
    gps_pred_stock = np.squeeze(np.argmax(soft_logits_gps_stock, axis = 1))
    gps_out_stock.extend(gps_pred_stock)
    gps_score_stock.extend(sentiment_score_gps_stock)
#print(gps_out) 
print(sentiment_score_gps_stock)

[-0.16555625 -0.16555625 -0.16555625 -0.13071045 -0.16555625 -0.16555625
 -0.16555625]


In [12]:
print(len(gps_out_stock))
gps_class_stock = [label_dict[pred] for pred in gps_out_stock]
print(len(gps_class_stock))
#print(gps_class)

507
507


In [13]:
label_dict = {0: 'positive', 1: 'negative', 2: 'neutral'}

gps_out_manage = []
gps_score_manage = []
for i in range(0, len(gps_manage_merge2), 10):
    tokenized_gps_manage = tokenizer(gps_manage_merge2[i:i+10], truncation = True, padding = True, return_tensors = "pt")
    gps_output_manage = model(tokenized_gps_manage['input_ids'])
    soft_logits_gps_manage = softmax(np.array(gps_output_manage.logits.detach().cpu()))
    sentiment_score_gps_manage = soft_logits_gps_manage[:,0]-soft_logits_gps_manage[:,1]
    gps_pred_manage = np.squeeze(np.argmax(soft_logits_gps_manage, axis = 1))
    gps_out_manage.extend(gps_pred_manage)
    gps_score_manage.extend(sentiment_score_gps_manage)
#print(gps_out) 
print(sentiment_score_gps_manage)

[-0.18687296 -0.18687293 -0.18687296 -0.18687293 -0.16651362 -0.1270847
 -0.19531702 -0.10084215 -0.18687296 -0.09479496]


In [14]:
print(len(gps_out_manage))
gps_class_manage = [label_dict[pred] for pred in gps_out_manage]
print(len(gps_class_manage))

120
120


In [15]:
label_dict = {0: 'positive', 1: 'negative', 2: 'neutral'}

gps_out_sales = []
gps_score_sales = []
for i in range(0, len(gps_sales_merge2), 10):
    tokenized_gps_sales = tokenizer(gps_sales_merge2[i:i+10], truncation = True, padding = True, return_tensors = "pt")
    gps_output_sales = model(tokenized_gps_sales['input_ids'])
    soft_logits_gps_sales = softmax(np.array(gps_output_sales.logits.detach().cpu()))
    sentiment_score_gps_sales = soft_logits_gps_sales[:,0]-soft_logits_gps_sales[:,1]
    gps_pred_sales = np.squeeze(np.argmax(soft_logits_gps_sales, axis = 1))
    gps_out_sales.extend(gps_pred_sales)
    gps_score_sales.extend(sentiment_score_gps_sales)
print(gps_out_sales) 
print(sentiment_score_gps_sales)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [16]:
print(len(gps_out_sales))
gps_class_sales = [label_dict[pred] for pred in gps_out_sales]
print(len(gps_class_sales))

1153
1153


In [17]:
label_dict = {0: 'positive', 1: 'negative', 2: 'neutral'}

gps_out_performance = []
gps_score_performance = []
for i in range(0, len(gps_performance_merge2), 10):
    tokenized_gps_performance = tokenizer(gps_performance_merge2[i:i+10], truncation = True, padding = True, return_tensors = "pt")
    gps_output_performance = model(tokenized_gps_performance['input_ids'])
    soft_logits_gps_performance = softmax(np.array(gps_output_performance.logits.detach().cpu()))
    sentiment_score_gps_performance = soft_logits_gps_performance[:,0]-soft_logits_gps_performance[:,1]
    gps_pred_performance = np.squeeze(np.argmax(soft_logits_gps_performance, axis = 1))
    gps_out_performance.extend(gps_pred_performance)
    gps_score_performance.extend(sentiment_score_gps_performance)
#print(gps_out_performance) 
print(sentiment_score_gps_performance)

[-0.09619686 -0.12438604]


In [18]:
print(len(gps_out_performance))
gps_class_performance = [label_dict[pred] for pred in gps_out_performance]
print(len(gps_class_performance))

262
262


In [19]:
gps_stock_price  = yf.download('gps', start='2006-11-30', end='2022-02-15', time_interval = 'daily')

[*********************100%***********************]  1 of 1 completed


In [20]:
gps_stock_price

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2006-11-29,19.030001,19.490000,18.959999,19.070000,12.795540,6591000
2006-11-30,18.500000,18.930000,18.400000,18.719999,12.560698,10621500
2006-12-01,18.730000,19.059999,18.559999,18.680000,12.533863,7224700
2006-12-04,18.719999,19.040001,18.580000,18.959999,12.721730,2819100
2006-12-05,18.900000,19.059999,18.740000,18.889999,12.674767,3362800
...,...,...,...,...,...,...
2022-02-08,17.000000,17.580000,16.870001,17.379999,16.892712,6194500
2022-02-09,17.480000,17.540001,17.190001,17.340000,16.853834,8041200
2022-02-10,17.190001,17.879999,17.139999,17.270000,16.785797,7120600


In [21]:

gps_stock_price = gps_stock_price.drop(['Open', 'High','Low','Volume', 'Adj Close'], axis = 1)

In [22]:
gps_stock_price

,Close
Date,
2006-11-29,19.070000
2006-11-30,18.719999
2006-12-01,18.680000
2006-12-04,18.959999
2006-12-05,18.889999
...,...
2022-02-08,17.379999
2022-02-09,17.340000
2022-02-10,17.270000


In [23]:
scaler = MinMaxScaler(feature_range=(0, 1))
gps_scaler_price = scaler.fit_transform(gps_stock_price.values.reshape(-1,1))

In [24]:
gps_manage_date = gps_manage_merge['Date']

In [25]:
gps_sales_date = gps_sales_merge['Date']
gps_performance_date = gps_performance_merge['Date']
gps_stock_date = gps_stock_merge['Date']


In [26]:
gps_score_stock = pd.DataFrame (gps_score_stock, columns = ['sentiment_score'])
gps_score_manage = pd.DataFrame (gps_score_stock, columns = ['sentiment_score'])
gps_score_performance = pd.DataFrame (gps_score_stock, columns = ['sentiment_score'])
gps_score_sales = pd.DataFrame (gps_score_stock, columns = ['sentiment_score'])


In [27]:
scaler = MinMaxScaler(feature_range=(0, 1))
gps_stock_scaler = scaler.fit_transform(gps_score_stock.values.reshape(-1,1))
gps_manage_scaler = scaler.fit_transform(gps_score_manage.values.reshape(-1,1))
gps_performance_scaler = scaler.fit_transform(gps_score_performance.values.reshape(-1,1))
gps_sales_scaler = scaler.fit_transform(gps_score_sales.values.reshape(-1,1))


In [28]:
gps_stock_scaler = pd.DataFrame (gps_stock_scaler, columns = ['sentiment_score'])
gps_manage_scaler = pd.DataFrame (gps_manage_scaler, columns = ['sentiment_score'])
gps_performance_scaler = pd.DataFrame (gps_performance_scaler, columns = ['sentiment_score'])
gps_sales_scaler = pd.DataFrame (gps_sales_scaler, columns = ['sentiment_score'])


In [29]:
gps_topic_stock_table = pd.concat([gps_stock_date, gps_stock_scaler], axis=1)
gps_topic_manage_table = pd.concat([gps_manage_date, gps_manage_scaler], axis=1)
gps_topic_performance_table = pd.concat([gps_performance_date, gps_performance_scaler], axis=1)
gps_topic_sales_table = pd.concat([gps_sales_date, gps_sales_scaler], axis=1)
    


In [30]:
print(len(gps_topic_stock_table))

gps_topic_stock_table.isnull().values.any()

507


False

In [31]:
print(len(gps_topic_manage_table))

gps_topic_manage_table.isnull().values.any()

507


True

In [32]:
print(len(gps_topic_performance_table))

gps_topic_performance_table.isnull().values.any()

507


True

In [33]:
print(len(gps_topic_sales_table))

gps_topic_sales_table.isnull().values.any()

1153


True

In [34]:
gps_stock_topic = gps_topic_stock_table.sort_values(by ="Date")
gps_manage_topic = gps_topic_manage_table.sort_values(by ="Date")
gps_sales_topic = gps_topic_sales_table.sort_values(by ="Date")
gps_performance_topic = gps_topic_performance_table.sort_values(by ="Date")


In [35]:
gps_stock_topic

,Date,sentiment_score
506,2007-01-04 06:01:00,0.332163
505,2007-02-26 07:01:00,0.332163
504,2007-03-23 01:01:00,0.332163
502,2007-04-12 07:01:00,0.332163
503,2007-04-12 11:23:00,0.499547
...,...,...
4,2022-02-08 04:56:00,0.495618
3,2022-02-09 04:56:00,0.751621
2,2022-02-10 04:57:00,0.623658
1,2022-02-11 04:56:00,0.745739


In [36]:
gps_manage_topic

,Date,sentiment_score
119,2006-12-06 10:38:00,0.718060
118,2007-01-04 06:01:00,0.858815
117,2007-01-22 05:28:00,0.764477
116,2007-01-22 07:31:00,0.515424
115,2007-01-23 11:14:00,0.242466
...,...,...
502,NaN,0.332163
503,NaN,0.499547
504,NaN,0.332163
505,NaN,0.332163


In [37]:
gps_performance_topic

,Date,sentiment_score
259,2007-02-08 04:41:00,0.751872
258,2007-03-01 06:30:00,0.822048
257,2007-03-02 04:40:00,0.822603
256,2007-05-02 04:39:00,0.580460
255,2007-05-24 06:35:00,0.714709
...,...,...
502,NaN,0.332163
503,NaN,0.499547
504,NaN,0.332163
505,NaN,0.332163


In [38]:
gps_sales_topic

,Date,sentiment_score
1149,2006-11-30 04:20:00,NaN
1148,2006-11-30 04:27:00,NaN
1147,2006-11-30 04:42:00,NaN
1146,2006-12-07 06:23:00,NaN
1145,2006-12-15 04:52:00,NaN
...,...,...
3,2022-02-01 04:57:00,0.751621
2,2022-02-02 04:57:00,0.623658
1150,2022-02-03 09:30:00,NaN
1,2022-02-07 04:57:00,0.745739


In [39]:
gps_stock_topic['Date'] = pd.to_datetime(gps_stock_topic['Date'])

In [40]:
gps_sales_topic['Date'] = pd.to_datetime(gps_sales_topic['Date'])
gps_manage_topic['Date'] = pd.to_datetime(gps_manage_topic['Date'])
gps_performance_topic['Date'] = pd.to_datetime(gps_performance_topic['Date'])


In [41]:
gps_stock_topic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 507 entries, 506 to 0
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             507 non-null    datetime64[ns]
 1   sentiment_score  507 non-null    float32       
dtypes: datetime64[ns](1), float32(1)
memory usage: 9.9 KB


In [42]:
gps_pure_price = pd.DataFrame(gps_scaler_price, columns = ['Closing_price'])


In [43]:
gps_stock_price1 = gps_stock_price.reset_index()


In [44]:
gps_stockprice_date = gps_stock_price1["Date"]


In [45]:
gps_stockprice_date = pd.DataFrame({ 'Stock_price_date':gps_stockprice_date.values})


In [46]:
gps_stock_table =pd.concat([gps_stockprice_date,gps_pure_price], axis=1)


In [47]:
print(gps_stock_table.isnull().values.any())


False


In [48]:
gps_stock_table1 = gps_stock_table.set_index('Stock_price_date')



In [49]:
gps_stock_table1


,Closing_price
Stock_price_date,
2006-11-29,0.330251
2006-11-30,0.321733
2006-12-01,0.320759
2006-12-04,0.327574
2006-12-05,0.325870
...,...
2022-02-08,0.289121
2022-02-09,0.288148
2022-02-10,0.286444


In [50]:
gps_stock_topic2 = gps_stock_topic.sort_values("Date")


In [51]:
gps_manage_topic2 = gps_manage_topic.sort_values("Date")
gps_performance_topic2 = gps_performance_topic.sort_values("Date")
gps_sales_topic2 = gps_sales_topic.sort_values("Date")


In [52]:
gps_stock_topic3 =  gps_stock_topic2.set_index('Date')


In [53]:
gps_manage_topic3 =  gps_manage_topic2.set_index('Date')
gps_sales_topic3 =  gps_sales_topic2.set_index('Date')
gps_performance_topic3 =  gps_performance_topic2.set_index('Date')


In [54]:
gps_stock_topic3.head()


,sentiment_score
Date,
2007-01-04 06:01:00,0.332163
2007-02-26 07:01:00,0.332163
2007-03-23 01:01:00,0.332163
2007-04-12 07:01:00,0.332163
2007-04-12 11:23:00,0.499547


In [55]:
gps_stock_topic3.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 507 entries, 2007-01-04 06:01:00 to 2022-02-14 04:56:00
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sentiment_score  507 non-null    float32
dtypes: float32(1)
memory usage: 5.9 KB


In [56]:
gps_stock_table1.head()



,Closing_price
Stock_price_date,
2006-11-29,0.330251
2006-11-30,0.321733
2006-12-01,0.320759
2006-12-04,0.327574
2006-12-05,0.325870


In [57]:
gps_stock_topic_final = gps_stock_topic3.assign(key=gps_stock_topic3.index.normalize())\
   .merge(gps_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)

C:\Users\User\AppData\Local\Temp\ipykernel_31676\517117334.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .merge(gps_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)


In [58]:
gps_manage_topic_final = gps_manage_topic3.assign(key=gps_manage_topic3.index.normalize())\
   .merge(gps_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)
gps_sales_topic_final = gps_sales_topic3.assign(key=gps_sales_topic3.index.normalize())\
   .merge(gps_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)
gps_performance_topic_final = gps_performance_topic3.assign(key=gps_performance_topic3.index.normalize())\
   .merge(gps_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)

C:\Users\User\AppData\Local\Temp\ipykernel_31676\564576651.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .merge(gps_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)
C:\Users\User\AppData\Local\Temp\ipykernel_31676\564576651.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .merge(gps_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)
C:\Users\User\AppData\Local\Temp\ipykernel_31676\564576651.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .merge(gps_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)


In [59]:
gps_stock_topic_final

,sentiment_score,Closing_price
Date,,
2007-01-04 06:01:00,0.332163,0.339255
2007-02-26 07:01:00,0.332163,0.344366
2007-03-23 01:01:00,0.332163,0.300560
2007-04-12 07:01:00,0.332163,0.319056
2007-04-12 11:23:00,0.499547,0.319056
...,...,...
2022-02-08 04:56:00,0.495618,0.289121
2022-02-09 04:56:00,0.751621,0.288148
2022-02-10 04:57:00,0.623658,0.286444


In [60]:
gps_manage_topic_final

,sentiment_score,Closing_price
Date,,
2006-12-06 10:38:00,0.718060,0.326844
2007-01-04 06:01:00,0.858815,0.339255
2007-01-22 05:28:00,0.764477,0.350450
2007-01-22 07:31:00,0.515424,0.350450
2007-01-23 11:14:00,0.242466,0.334875
...,...,...
NaT,0.332163,NaN
NaT,0.499547,NaN
NaT,0.332163,NaN


In [66]:
gps_stock_topic_lstm = gps_stock_topic_final.dropna()
gps_manage_topic_lstm = gps_manage_topic_final.dropna()
gps_performance_topic_lstm = gps_performance_topic_final.dropna()
gps_sales_topic_lstm = gps_sales_topic_final.dropna()


In [67]:
gps_stock_topic_lstm.to_csv('gps_stock_topic_lstm_bert.csv', index = True)
gps_performance_topic_lstm.to_csv('gps_performance_topic_lstm_bert.csv', index = True)
gps_sales_topic_lstm.to_csv('gps_sales_topic_lstm_bert.csv', index = True)
gps_manage_topic_lstm.to_csv('gps_manage_topic_lstm_bert.csv', index = True)

In [69]:
gps_stock_topic_lstm_ffill = gps_stock_topic_final.fillna(method= 'bfill')
gps_manage_topic_lstm_ffill = gps_manage_topic_final.fillna(method='bfill')
gps_performance_topic_lstm_ffill = gps_performance_topic_final.fillna(method = 'bfill')
gps_sales_topic_lstm_ffill = gps_sales_topic_final.fillna(method='bfill')

In [65]:
gps_stock_topic_lstm_ffill.to_csv('gps_stock_topic_lstm_ffill_bert.csv', index = True)
gps_performance_topic_lstm_ffill.to_csv('gps_performance_topic_lstm_ffill_bert.csv', index = True)
gps_sales_topic_lstm_ffill.to_csv('gps_sales_topic_lstm_ffill_bert.csv', index = True)
gps_manage_topic_lstm_ffill.to_csv('gps_manage_topic_lstm_ffill_bert.csv', index = True)

In [71]:
len(gps_sales_topic_lstm_ffill)

1153

In [72]:
len(gps_sales_topic_lstm)

487